In [4]:
import numpy as np
import pandapower as pp

# Crear la red de ejemplo
def crear_red():
    net = pp.create_empty_network()

    # Crear barras
    bus1 = pp.create_bus(net, vn_kv=110, name="Bus 1")
    bus2 = pp.create_bus(net, vn_kv=220, name="Bus 2")
    bus1a = pp.create_bus(net, vn_kv=220, name="Bus 1A")
    bus2a = pp.create_bus(net, vn_kv=220, name="Bus 2A")
    bus3a = pp.create_bus(net, vn_kv=220, name="Bus 3A")
    bus1b = pp.create_bus(net, vn_kv=220, name="Bus 1B")
    bus2b = pp.create_bus(net, vn_kv=220, name="Bus 2B")

    # Crear las líneas de transmisión
    pp.create_line(net, from_bus=bus2, to_bus=bus1a, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 2-1A")
    pp.create_line(net, from_bus=bus1a, to_bus=bus2a, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 1A-2A")
    pp.create_line(net, from_bus=bus2a, to_bus=bus3a, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 2A-3A")
    pp.create_line(net, from_bus=bus3a, to_bus=bus2b, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 3A-2B")
    pp.create_line(net, from_bus=bus2, to_bus=bus1b, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 2-1B")
    pp.create_line(net, from_bus=bus1b, to_bus=bus2b, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 1B-2B")

    # Crear un transformador
    pp.create_transformer(net, hv_bus=bus2, lv_bus=bus1, std_type="100 MVA 220/110 kV", name="Trafo 1-2")

    # Crear una barra slack
    pp.create_ext_grid(net, bus=bus1, vm_pu=1.0)

    # Definir la carga nominal para todas las barras
    p_nominal = 150  # MW
    q_nominal = 100  # MVAr

    # Cargas para cada barra 
    pp.create_load(net, bus=bus1a, p_mw=p_nominal*0.2, q_mvar=q_nominal*0.2, name="Load 1A")
    pp.create_load(net, bus=bus2a, p_mw=p_nominal*0.35, q_mvar=q_nominal*0.35, name="Load 2A")
    pp.create_load(net, bus=bus3a, p_mw=p_nominal*0.15, q_mvar=q_nominal*0.15, name="Load 3A")
    pp.create_load(net, bus=bus1b, p_mw=p_nominal*0.1, q_mvar=q_nominal*0.1, name="Load 1B")
    pp.create_load(net, bus=bus2b, p_mw=p_nominal*0.6, q_mvar=q_nominal*0.6, name="Load 2B")

    return net
# Ejecutar el flujo de potencia
pp.runpp(net)

# Obtener la matriz Ybus
Ybus_dense = net["_ppc"]["internal"]["Ybus"].toarray()

# Definir P y Q especificados para todas las barras excepto la slack
P_spec = np.array([0, 30, 52.5, 22.5, 15, 10, 90])  # Potencias activas especificadas (P) en MW
Q_spec = np.array([0, 20, 35, 15, 10, 5, 60])      # Potencias reactivas especificadas (Q) en MVAr

# Ejecutar el método de Newton-Raphson
V, theta, iterations = newton_raphson(Ybus_dense, P_spec, Q_spec)

def newton_raphson(Ybus_dense, P_spec, Q_spec, max_iter=10, tol=1e-6):
    num_buses = Ybus_dense.shape[0]
    
    # Inicializar tensiones y ángulos
    V = np.ones(num_buses)  # magnitudes de tensión inicializadas a 1.0 p.u.
    theta = np.zeros(num_buses)  # ángulos de tensión inicializados a 0.0 radianes

    def calculate_PQ(Ybus, V, theta):
        P = np.zeros(len(V))
        Q = np.zeros(len(V))
        for i in range(len(V)):
            for j in range(len(V)):
                P[i] += V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                Q[i] += V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
        return P, Q

    def create_jacobian(Ybus, V, theta):
        num_buses = len(V)
        J1 = np.zeros((num_buses-1, num_buses-1))
        J2 = np.zeros((num_buses-1, num_buses-1))
        J3 = np.zeros((num_buses-1, num_buses-1))
        J4 = np.zeros((num_buses-1, num_buses-1))
        for i in range(1, num_buses):
            for j in range(1, num_buses):
                if i == j:
                    for k in range(num_buses):
                        J1[i-1, j-1] += V[i] * V[k] * (-Ybus[i, k].real * np.sin(theta[i] - theta[k]) + Ybus[i, k].imag * np.cos(theta[i] - theta[k]))
                        J2[i-1, j-1] += V[k] * (Ybus[i, k].real * np.cos(theta[i] - theta[k]) + Ybus[i, k].imag * np.sin(theta[i] - theta[k]))
                        J3[i-1, j-1] += V[i] * V[k] * (Ybus[i, k].real * np.cos(theta[i] - theta[k]) + Ybus[i, k].imag * np.sin(theta[i] - theta[k]))
                        J4[i-1, j-1] += V[k] * (Ybus[i, k].real * np.sin(theta[i] - theta[k]) - Ybus[i, k].imag * np.cos(theta[i] - theta[k]))
                    J2[i-1, j-1] += 2 * V[i] * Ybus[i, i].real
                    J4[i-1, j-1] -= 2 * V[i] * Ybus[i, i].imag
                else:
                    J1[i-1, j-1] += V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
                    J2[i-1, j-1] += V[i] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                    J3[i-1, j-1] += V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                    J4[i-1, j-1] += V[i] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
        return J1, J2, J3, J4

    # Iteraciones de Newton-Raphson
    for iteration in range(max_iter):
        P, Q = calculate_PQ(Ybus_dense, V, theta)
        
        dP = P_spec[1:] - P[1:]
        dQ = Q_spec[1:] - Q[1:]
        
        mismatch = np.concatenate((dP, dQ))
        
        if np.max(np.abs(mismatch)) < tol:
            break
        
        J1, J2, J3, J4 = create_jacobian(Ybus_dense, V, theta)
        J = np.block([[J1, J2], [J3, J4]])
        
        correction = np.linalg.solve(J, mismatch)
        
        theta[1:] += correction[:num_buses-1]
        V[1:] += correction[num_buses-1:]
    
    return V, theta, iteration



print("Tensiones finales (algoritmo Newton-Raphson):")
print(V)
print("Ángulos finales (algoritmo Newton-Raphson):")
print(theta)
print("Número de iteraciones (algoritmo Newton-Raphson):", iterations)
print("Número de iteraciones PandaPower", net._ppc['iterations'])

# Resultados de pandapower
V_pp = net.res_bus.vm_pu
theta_pp = net.res_bus.va_degree

print("\nTensiones finales (pandapower):")
print(V_pp)
print("Ángulos finales (pandapower):")
print(theta_pp)

# Comparación detallada
print("\nDiferencias en tensiones:")
for i in range(len(V)):
    print(f"Bus {i+1}: Newton-Raphson = {V[i]:.6f} p.u., PandaPower = {V_pp[i]:.6f} p.u., Diferencia = {V[i] - V_pp[i]:.6f} p.u.")

print("\nDiferencias en ángulos:")
for i in range(len(theta)):
    print(f"Bus {i+1}: Newton-Raphson = {theta[i]:.6f} rad, PandaPower = {np.radians(theta_pp[i]):.6f} rad, Diferencia = {theta[i] - np.radians(theta_pp[i]):.6f} rad")


NameError: name 'net' is not defined

In [7]:
import numpy as np
import pandapower as pp

# Crear la red de ejemplo
def crear_red():
    net = pp.create_empty_network()

    # Crear barras
    bus1 = pp.create_bus(net, vn_kv=110, name="Bus 1")
    bus2 = pp.create_bus(net, vn_kv=220, name="Bus 2")
    bus1a = pp.create_bus(net, vn_kv=220, name="Bus 1A")
    bus2a = pp.create_bus(net, vn_kv=220, name="Bus 2A")
    bus3a = pp.create_bus(net, vn_kv=220, name="Bus 3A")
    bus1b = pp.create_bus(net, vn_kv=220, name="Bus 1B")
    bus2b = pp.create_bus(net, vn_kv=220, name="Bus 2B")

    # Crear las líneas de transmisión
    pp.create_line(net, from_bus=bus2, to_bus=bus1a, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 2-1A")
    pp.create_line(net, from_bus=bus1a, to_bus=bus2a, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 1A-2A")
    pp.create_line(net, from_bus=bus2a, to_bus=bus3a, length_km=20, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 2A-3A")
    pp.create_line(net, from_bus=bus3a, to_bus=bus2b, length_km=15, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 3A-2B")
    pp.create_line(net, from_bus=bus2, to_bus=bus1b, length_km=10, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 2-1B")
    pp.create_line(net, from_bus=bus1b, to_bus=bus2b, length_km=30, std_type="N2XS(FL)2Y 1x185 RM/35 64/110 kV", name="Linea 1B-2B")

    # Crear un transformador
    pp.create_transformer(net, hv_bus=bus2, lv_bus=bus1, std_type="100 MVA 220/110 kV", name="Trafo 1-2")

    # Crear una barra slack
    pp.create_ext_grid(net, bus=bus1, vm_pu=1.0)

    # Definir la carga nominal para todas las barras
    p_nominal = 150  # MW
    q_nominal = 100  # MVAr

    # Cargas para cada barra 
    pp.create_load(net, bus=bus1a, p_mw=p_nominal*0.2, q_mvar=q_nominal*0.2, name="Load 1A")
    pp.create_load(net, bus=bus2a, p_mw=p_nominal*0.35, q_mvar=q_nominal*0.35, name="Load 2A")
    pp.create_load(net, bus=bus3a, p_mw=p_nominal*0.15, q_mvar=q_nominal*0.15, name="Load 3A")
    pp.create_load(net, bus=bus1b, p_mw=p_nominal*0.1, q_mvar=q_nominal*0.1, name="Load 1B")
    pp.create_load(net, bus=bus2b, p_mw=p_nominal*0.6, q_mvar=q_nominal*0.6, name="Load 2B")

    return net

def newton_raphson(Ybus_dense, P_spec, Q_spec, max_iter=10, tol=1e-6):
    num_buses = Ybus_dense.shape[0]
    
    # Inicializar tensiones y ángulos
    V = np.ones(num_buses)  # magnitudes de tensión inicializadas a 1.0 p.u.
    theta = np.zeros(num_buses)  # ángulos de tensión inicializados a 0.0 radianes

    def calculate_PQ(Ybus, V, theta):
        P = np.zeros(len(V))
        Q = np.zeros(len(V))
        for i in range(len(V)):
            for j in range(len(V)):
                P[i] += V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                Q[i] += V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
        return P, Q

    def create_jacobian(Ybus, V, theta):
        num_buses = len(V)
        J1 = np.zeros((num_buses-1, num_buses-1))
        J2 = np.zeros((num_buses-1, num_buses-1))
        J3 = np.zeros((num_buses-1, num_buses-1))
        J4 = np.zeros((num_buses-1, num_buses-1))
        for i in range(1, num_buses):
            for j in range(1, num_buses):
                if i == j:
                    for k in range(num_buses):
                        J1[i-1, j-1] += V[i] * V[k] * (-Ybus[i, k].real * np.sin(theta[i] - theta[k]) + Ybus[i, k].imag * np.cos(theta[i] - theta[k]))
                        J2[i-1, j-1] += V[k] * (Ybus[i, k].real * np.cos(theta[i] - theta[k]) + Ybus[i, k].imag * np.sin(theta[i] - theta[k]))
                        J3[i-1, j-1] += V[i] * V[k] * (Ybus[i, k].real * np.cos(theta[i] - theta[k]) + Ybus[i, k].imag * np.sin(theta[i] - theta[k]))
                        J4[i-1, j-1] += V[k] * (Ybus[i, k].real * np.sin(theta[i] - theta[k]) - Ybus[i, k].imag * np.cos(theta[i] - theta[k]))
                    J2[i-1, j-1] += 2 * V[i] * Ybus[i, i].real
                    J4[i-1, j-1] -= 2 * V[i] * Ybus[i, i].imag
                else:
                    J1[i-1, j-1] += V[i] * V[j] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
                    J2[i-1, j-1] += V[i] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                    J3[i-1, j-1] += V[i] * V[j] * (Ybus[i, j].real * np.cos(theta[i] - theta[j]) + Ybus[i, j].imag * np.sin(theta[i] - theta[j]))
                    J4[i-1, j-1] += V[i] * (Ybus[i, j].real * np.sin(theta[i] - theta[j]) - Ybus[i, j].imag * np.cos(theta[i] - theta[j]))
        return J1, J2, J3, J4

    # Iteraciones de Newton-Raphson
    for iteration in range(max_iter):
        P, Q = calculate_PQ(Ybus_dense, V, theta)
        
        dP = P_spec[1:] - P[1:]
        dQ = Q_spec[1:] - Q[1:]
        
        mismatch = np.concatenate((dP, dQ))
        
        if np.max(np.abs(mismatch)) < tol:
            break
        
        J1, J2, J3, J4 = create_jacobian(Ybus_dense, V, theta)
        J = np.block([[J1, J2], [J3, J4]])
        
        correction = np.linalg.solve(J, mismatch)
        
        theta[1:] += correction[:num_buses-1]
        V[1:] += correction[num_buses-1:]
    
    return V, theta, iteration

# Crear la red y ejecutar el flujo de potencia
net = crear_red()
pp.runpp(net)

# Obtener la matriz Ybus
Ybus_dense = net["_ppc"]["internal"]["Ybus"].toarray()

# Definir P y Q especificados para todas las barras excepto la slack
P_spec = np.array([0, 30, 52.5, 22.5, 15, 10, 90])  # Potencias activas especificadas (P) en MW
Q_spec = np.array([0, 20, 35, 15, 10, 5, 60])      # Potencias reactivas especificadas (Q) en MVAr

# Ejecutar el método de Newton-Raphson
V, theta, iterations = newton_raphson(Ybus_dense, P_spec, Q_spec)

print("Tensiones finales (algoritmo Newton-Raphson):")
print(V)
print("Ángulos finales (algoritmo Newton-Raphson):")
print(theta)
print("Número de iteraciones (algoritmo Newton-Raphson):", iterations)
print("Número de iteraciones PandaPower:", net._ppc['iterations'])

# Resultados de pandapower
V_pp = net.res_bus.vm_pu
theta_pp = net.res_bus.va_degree

print("\nTensiones finales (pandapower):")
print(V_pp)
print("Ángulos finales (pandapower):")
print(theta_pp)

# Comparación detallada
print("\nDiferencias en tensiones:")
for i in range(len(V)):
    print(f"Bus {i+1}: Newton-Raphson = {V[i]:.6f} p.u., PandaPower = {V_pp[i]:.6f} p.u., Diferencia = {V[i] - V_pp[i]:.6f} p.u.")

print("\nDiferencias en ángulos:")
for i in range(len(theta)):
    print(f"Bus {i+1}: Newton-Raphson = {theta[i]:.6f} rad, PandaPower = {np.radians(theta_pp[i]):.6f} rad, Diferencia = {theta[i] - np.radians(theta_pp[i]):.6f} rad")


Tensiones finales (algoritmo Newton-Raphson):
[1.         0.12993432 0.05822993 0.06211078 0.06819998 0.21959465
 0.06730054]
Ángulos finales (algoritmo Newton-Raphson):
[ 0.          1.7622077  -1.29594382  1.42786467  1.39620249  0.89498234
 -1.73311715]
Número de iteraciones (algoritmo Newton-Raphson): 9
Número de iteraciones PandaPower: 4

Tensiones finales (pandapower):
0    1.000000
1    1.033380
2    1.031457
3    1.029237
4    1.028365
5    1.032375
6    1.027435
Name: vm_pu, dtype: float64
Ángulos finales (pandapower):
0     0.000000
1   -14.276471
2   -14.492303
3   -14.730336
4   -14.864945
5   -14.469768
6   -14.873522
Name: va_degree, dtype: float64

Diferencias en tensiones:
Bus 1: Newton-Raphson = 1.000000 p.u., PandaPower = 1.000000 p.u., Diferencia = 0.000000 p.u.
Bus 2: Newton-Raphson = 0.129934 p.u., PandaPower = 1.033380 p.u., Diferencia = -0.903446 p.u.
Bus 3: Newton-Raphson = 0.058230 p.u., PandaPower = 1.031457 p.u., Diferencia = -0.973227 p.u.
Bus 4: Newton-Raph